> # **TensorFlow  2.1.0** - ***10*** - *Tensorflow Lite*

In [1]:
!pip install tensorflow-gpu==1.13.1
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [0]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist

In [0]:
# carregando base de dados
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# normalizando base de dados
X_train = X_train / 255.
X_test = X_test / 255.

# mudando a dimensionalidade da base de dados
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [4]:
# construindo o modelo
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
# compilando o modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [5]:
# treinando o modelo
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 6s 107us/sample - loss: 0.5301 - sparse_categorical_accuracy: 0.8138
Epoch 2/5
60000/60000 [==============================] - 6s 96us/sample - loss: 0.3990 - sparse_categorical_accuracy: 0.8548
Epoch 3/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3669 - sparse_categorical_accuracy: 0.8670
Epoch 4/5
60000/60000 [==============================] - 6s 101us/sample - loss: 0.3451 - sparse_categorical_accuracy: 0.8726
Epoch 5/5
60000/60000 [==============================] - 5s 91us/sample - loss: 0.3287 - sparse_categorical_accuracy: 0.8792


In [6]:
# avaliando o modelo
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 50us/sample - loss: 0.3668 - sparse_categorical_accuracy: 0.8673
Test accuracy: 0.8672999739646912


In [7]:
# salvando e convertendo o modelo para Tensorflow Lite
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)
tflite_model = converter.convert()

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


In [0]:
with open("tflite_model", 'wb') as f:
  f.write(tflite_model)